In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from pylab import rcParams
from sklearn.utils import shuffle
from sklearn.feature_selection import VarianceThreshold 

In [149]:
biodeg = pd.read_csv('../../big datasets/featurized_biodeg.csv', low_memory=False)

In [150]:
pd.options.mode.chained_assignment = None
train_x = biodeg[biodeg['Status'].str.contains('Train')]
bin_class = []

for index, row in train_x.iterrows():
    if row['Class'] == 'RB':
        bin_class.append(1)
    else:
        bin_class.append(0)
train_x.drop(['Class'], axis=1)
train_x['Class'] = bin_class
train_y = train_x['Class']
train_x.drop(train_x.select_dtypes(['object']), inplace=True, axis=1)
train_x.head()

,Unnamed: 0,Class,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
0,0,1,0.0,-2.4894,6.197112,38.0018,35.363032,0.0,0,35,...,4.462509,20.828762,1.893524,0.000000,0.000000,0.0,220,8,6.599,38
1,1,1,0.0,-0.4296,0.184556,18.5395,13.176344,0.0,0,13,...,5.542886,9.148995,1.829799,2.705816,2.705816,0.0,17,2,0.628,22
2,2,1,0.0,-0.5003,0.250300,45.6856,29.290274,0.0,0,30,...,5.937350,22.828595,1.902383,11.163248,11.163248,0.0,286,9,-0.646,42
3,3,1,1.0,-0.3087,0.095296,8.5717,4.697586,0.0,0,5,...,9.201096,4.914214,1.638071,4.414214,4.414214,0.0,4,0,-0.316,6
4,4,1,0.0,-0.8561,0.732907,50.2372,34.143860,0.0,0,34,...,6.004005,26.661622,1.904402,11.202922,11.202922,0.0,444,11,1.211,52


In [151]:
test_x = biodeg[biodeg['Status'].str.contains('Test')]
test_class = []

for index, row in test_x.iterrows():
    if row['Class'] == 'RB':
        test_class.append(1)
    else:
        test_class.append(0)
test_x.drop(['Class'], axis=1)
test_x['Class'] = test_class
test_y = test_x['Class']
test_x.drop(test_x.select_dtypes(['object']), inplace=True, axis=1)
test_x.head()

,Unnamed: 0,Class,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,...,AMW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
837,837,1,0.0,-1.3374,1.788639,26.3554,22.988688,0.0,0,23,...,4.353270,12.833789,1.833398,0.000000,0.000000,0.000000,56,4,4.323,22
838,838,1,0.0,-0.1309,0.017135,11.0435,6.989172,0.0,0,7,...,6.289459,5.250000,1.750000,2.500000,2.500000,0.000000,3,0,-0.190,12
839,839,1,0.0,1.8498,3.421760,35.2378,17.654344,0.0,0,15,...,6.137507,13.676825,1.953832,0.000000,0.000000,0.000000,42,5,2.474,30
840,840,1,0.0,-1.4466,2.092652,36.1682,27.849067,0.0,0,28,...,4.612562,16.829768,1.869974,2.926777,0.000000,2.926777,120,6,2.524,30
841,841,1,0.0,2.4692,6.096949,49.7536,30.404274,0.0,0,29,...,5.315026,20.254935,1.841358,2.400011,2.400011,0.000000,180,10,2.468,46


if 'Class' in train_x.columns:
    print('Yes')

drugall = drug_pos.append(drug_neg)

drugall.to_csv('../../big datasets/drugall_with_classes.csv')

drug_pos_features = drug_pos.iloc[:,2:]
drug_pos_features.head()
drug_pos_class = drug_pos['Class']

drug_neg_features = drug_neg.iloc[:,2:]
drug_neg_class = drug_neg['Class']

x_train_neg, x_test_neg, y_train_neg, y_test_neg = train_test_split(drug_neg_features, drug_neg_class, test_size=0.2, random_state=12)
x_train, x_test, y_train, y_test = train_test_split(drug_pos_features, drug_pos_class, test_size=0.2, random_state=12)


In [152]:
constant_filter = VarianceThreshold(threshold=0)
qconstant_filter = VarianceThreshold(threshold=0.01)

In [153]:
constant_filter.fit(train_x)
len(train_x.columns[constant_filter.get_support()])
constant_columns = [column for column in train_x.columns if column
                   not in train_x.columns[constant_filter.get_support()]]

train_x.drop(labels=constant_columns, axis=1, inplace=True)
test_x.drop(labels=constant_columns, axis=1, inplace=True)

In [154]:
qconstant_filter.fit(train_x)
len(train_x.columns[qconstant_filter.get_support()])

q_constant_columns = [column for column in train_x.columns if column
                     not in train_x.columns[qconstant_filter.get_support()]]

train_x = qconstant_filter.transform(train_x)
test_x = qconstant_filter.transform(test_x)

train_x.shape, test_x.shape

((837, 1012), (218, 1012))

In [155]:
correlated_features = set()
correlation_matrix = pd.DataFrame(train_x).corr()

In [156]:
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i,j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)

In [157]:
len(correlated_features)

716

In [158]:
train_x = pd.DataFrame(train_x)
test_x = pd.DataFrame(test_x)

In [159]:
train_x.drop(labels=correlated_features, axis=1, inplace=True)
test_x.drop(labels=correlated_features, axis=1, inplace=True)

In [160]:
train_x.head(10)


,0,2,3,4,5,11,12,13,14,15,...,935,939,941,942,943,945,951,957,968,995
0,0.0,0.0,-2.4894,6.197112,38.0018,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,1.000000,0.050000,0.000000
1,1.0,0.0,-0.4296,0.184556,18.5395,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.500000,0.200000,3.713572
2,2.0,0.0,-0.5003,0.250300,45.6856,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.833333,0.045455,0.000000
3,3.0,1.0,-0.3087,0.095296,8.5717,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.250000,0.000000
4,4.0,0.0,-0.8561,0.732907,50.2372,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.769231,1.000000,0.115385,0.000000
5,5.0,0.0,0.0097,0.000094,43.7936,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.727273,1.000000,0.136364,0.000000
6,6.0,1.0,-0.4392,0.192897,16.6161,0.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.750000,0.000000
7,7.0,0.0,-1.0933,1.195305,52.1502,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.692308,1.000000,0.192308,0.000000
8,8.0,0.0,2.2962,5.272534,39.5198,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.250000,0.000000
9,9.0,1.0,1.1813,1.395470,40.1727,1.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.666667,0.208333,0.000000


In [161]:
# only run this cell once
train_x.drop([train_x.columns[0]],  axis='columns')
test_x.drop([test_x.columns[0]],  axis='columns')

,2,3,4,5,11,12,13,14,15,16,...,935,939,941,942,943,945,951,957,968,995
0,0.0,-1.3374,1.788639,26.3554,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,1.000000,0.083333,0.000000
1,0.0,-0.1309,0.017135,11.0435,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.433987
2,0.0,1.8498,3.421760,35.2378,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.142857,0.000000
3,0.0,-1.4466,2.092652,36.1682,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.750000,1.000000,0.062500,0.000000
4,0.0,2.4692,6.096949,49.7536,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.400000,0.750000,0.400000,0.000000
5,0.0,-1.2352,1.525719,41.0857,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,1.000000,0.050000,0.000000
6,0.0,-0.3943,0.155472,19.6534,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,1.000000,0.125000,0.000000
7,0.0,-0.0853,0.007276,31.6522,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,1.000000,0.571429,0.000000
8,0.0,-0.1960,0.038416,35.9868,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.600000,1.000000,0.250000,0.000000
9,0.0,-1.5284,2.336007,87.7498,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.695652,0.888889,0.195652,0.000000


In [162]:
all_train = train_x
all_test = test_x

In [163]:
all_train.astype('float64')
all_train.drop(all_train.select_dtypes(['object']), inplace=True, axis=1)


all_test.drop(all_test.select_dtypes(['object']), inplace=True, axis=1)
all_test.astype('float64')

all_train.head(), all_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 837 entries, 0 to 836
Columns: 296 entries, 0 to 995
dtypes: float64(296)
memory usage: 1.9 MB


(   0    2       3         4        5    11   12   13   14   15   ...  935  \
 0  0.0  0.0 -2.4894  6.197112  38.0018  0.0  0.0  0.0  0.0  0.0  ...  0.0   
 1  1.0  0.0 -0.4296  0.184556  18.5395  0.0  1.0  0.0  0.0  0.0  ...  0.0   
 2  2.0  0.0 -0.5003  0.250300  45.6856  0.0  4.0  0.0  0.0  0.0  ...  0.0   
 3  3.0  1.0 -0.3087  0.095296   8.5717  0.0  2.0  0.0  0.0  0.0  ...  0.0   
 4  4.0  0.0 -0.8561  0.732907  50.2372  0.0  4.0  0.0  0.0  0.0  ...  0.0   
 
    939  941  942  943  945       951       957       968       995  
 0  0.0  0.0  0.0  0.0  0.0  0.800000  1.000000  0.050000  0.000000  
 1  1.0  0.0  0.0  0.0  0.0  0.200000  0.500000  0.200000  3.713572  
 2  0.0  0.0  0.0  0.0  0.0  0.818182  0.833333  0.045455  0.000000  
 3  0.0  0.0  0.0  0.0  0.0  0.000000  1.000000  0.250000  0.000000  
 4  0.0  0.0  0.0  0.0  0.0  0.769231  1.000000  0.115385  0.000000  
 
 [5 rows x 296 columns], None)

with pd.option_context('mode.use_inf_as_null', True):
    all_train = all_train.dropna(how='all', axis=1)
    all_test = all_test.dropna(how='all',axis=1)

In [164]:
all_train = all_train[np.isfinite(all_train).all(1)]
all_test = all_test[np.isfinite(all_test).all(1)]
all_train.shape[1] == all_test.shape[1]

True

In [165]:
y_train = all_train.iloc[:,1]
x_train = all_train.iloc[:,2:]

x_test = all_test.iloc[:,2:]
y_test = all_test.iloc[:,1]

In [166]:
x_train

,3,4,5,11,12,13,14,15,16,17,...,935,939,941,942,943,945,951,957,968,995
0,-2.4894,6.197112,38.0018,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.800000,1.000000,0.050000,0.000000
1,-0.4296,0.184556,18.5395,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.200000,0.500000,0.200000,3.713572
2,-0.5003,0.250300,45.6856,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.818182,0.833333,0.045455,0.000000
3,-0.3087,0.095296,8.5717,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.250000,0.000000
4,-0.8561,0.732907,50.2372,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.769231,1.000000,0.115385,0.000000
5,0.0097,0.000094,43.7936,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.727273,1.000000,0.136364,0.000000
6,-0.4392,0.192897,16.6161,0.0,3.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.750000,0.000000
7,-1.0933,1.195305,52.1502,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.692308,1.000000,0.192308,0.000000
8,2.2962,5.272534,39.5198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.333333,0.250000,0.000000
9,1.1813,1.395470,40.1727,1.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.666667,0.208333,0.000000


In [167]:
from sklearn.preprocessing import StandardScaler
xscaler = StandardScaler().fit(x_train)
x_train = xscaler.transform(x_train)
testscaler = StandardScaler().fit(x_test)
x_test = testscaler.transform(x_test)

In [168]:
x_train.shape[0] == y_train.shape[0]

True

In [169]:
sm = SMOTE(random_state=2)
x_train_res, y_train_res = sm.fit_resample(x_train, y_train)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [34]:
print(len(x_train),len(y_train),len(x_train_res),len(y_train_res))

837 837 1106 1106
